In [4]:
import sys, types
import numpy as np
import pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
import os, re
import numpy as np
import pandas as pd
from PIL import Image
from pathlib import Path
import anndata as ad
from shapely.geometry import Point, Polygon
import geopandas as gpd
import scanpy as sc

# Load Transcripts

In [3]:
affine_matrix = np.loadtxt('path/to/file/matrix.csv', delimiter=',')
affine_matrix_inv = np.linalg.inv(affine_matrix)


In [4]:
transcripts_path = "path/to/file/transcripts.parquet"

df_transcripts = pd.read_parquet(transcripts_path)


In [21]:
filtered_transcripts = df_transcripts[
    (df_transcripts['is_gene'] == True) & (df_transcripts['qv'] > 20)
]
filtered_transcripts['x_coord'] = filtered_transcripts['x_location']/0.2125
filtered_transcripts['y_coord'] = filtered_transcripts['y_location']/0.2125
homogeneous_coordinates = np.hstack([filtered_transcripts[['x_coord', 'y_coord']].values, np.ones((filtered_transcripts.shape[0], 1))])

transformed_coordinates = homogeneous_coordinates.dot(affine_matrix_inv.T)
filtered_transcripts['x_transformed'] = transformed_coordinates[:, 0]
filtered_transcripts['y_transformed'] = transformed_coordinates[:, 1]


In [24]:
filtered_transcripts

,transcript_id,cell_id,overlaps_nucleus,feature_name,x_location,y_location,z_location,qv,fov_name,nucleus_distance,codeword_index,codeword_category,is_gene,x_coord,y_coord,x_transformed,y_transformed
0,281930243282594,UNASSIGNED,0,ALCAM,76.859375,723.890625,27.703125,40.00,C2,263.203125,278,custom_gene,True,361.691162,3406.543945,-1951.850024,-3344.648369
1,281930243245238,UNASSIGNED,0,ASS1,89.625000,721.875000,27.703125,31.00,C2,251.390625,165,custom_gene,True,421.764679,3397.058838,-1912.556068,-3350.867741
2,281930243245251,UNASSIGNED,0,ASS1,138.812500,723.843750,27.734375,31.00,C2,203.812500,165,custom_gene,True,653.235291,3406.323486,-1761.140351,-3344.864473
3,281930243280311,UNASSIGNED,0,ASS1,128.031250,749.984375,27.812500,40.00,C2,207.500000,165,custom_gene,True,602.500000,3529.338135,-1794.297833,-3264.383560
4,281930243256169,UNASSIGNED,0,ATXN2,166.281250,717.031250,27.734375,33.00,C2,180.671875,112,custom_gene,True,782.500000,3374.264648,-1676.591460,-3365.867213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131578067,281578055958065,UNASSIGNED,0,BAX,10250.421875,19263.015625,14.015625,40.00,AB18,142.890625,215,custom_gene,True,48237.277344,90649.484375,29386.843931,53712.358921
131578069,281646775406687,UNASSIGNED,0,BAX,10250.937500,19730.250000,13.937500,31.00,AC18,112.437500,215,custom_gene,True,48239.703125,92848.234375,29388.972927,55150.641494
131578070,281646775430168,UNASSIGNED,0,YTHDF2,10250.781250,19712.109375,13.859375,40.00,AC18,115.046875,135,custom_gene,True,48238.968750,92762.867188,29388.471494,55094.799861
131578072,281646775430169,UNASSIGNED,0,YTHDF2,10250.640625,20099.406250,13.828125,25.25,AC18,236.515625,135,custom_gene,True,48238.308594,94585.437500,29388.489101,56287.010239


In [25]:
filtered_transcripts = filtered_transcripts[
    (filtered_transcripts['x_transformed'] > 0) &
    (filtered_transcripts['y_transformed'] > 0)
]

In [26]:
filtered_transcripts

,transcript_id,cell_id,overlaps_nucleus,feature_name,x_location,y_location,z_location,qv,fov_name,nucleus_distance,codeword_index,codeword_category,is_gene,x_coord,y_coord,x_transformed,y_transformed
415858,282003257711913,UNASSIGNED,0,AMFR,731.406250,1958.875000,26.843750,40.00,D3,104.875000,274,custom_gene,True,3441.911621,9218.235352,64.468506,456.232969
415859,282003257711914,UNASSIGNED,0,AMFR,731.484375,1957.500000,26.875000,40.00,D3,105.500000,274,custom_gene,True,3442.279297,9211.764648,64.707421,452.000154
416388,282003257688584,UNASSIGNED,0,BAX,726.328125,1996.578125,26.906250,40.00,D3,91.437500,215,custom_gene,True,3418.014648,9395.661133,48.880371,572.299586
416413,282003257701473,UNASSIGNED,0,BAX,740.109375,1858.171875,26.828125,40.00,D3,108.078125,215,custom_gene,True,3482.867432,8744.337891,91.142346,146.229126
416437,282003257709288,UNASSIGNED,0,C9orf72,714.640625,1899.843750,26.875000,40.00,D3,113.343750,367,custom_gene,True,3363.014648,8940.441406,12.790595,274.537197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131578067,281578055958065,UNASSIGNED,0,BAX,10250.421875,19263.015625,14.015625,40.00,AB18,142.890625,215,custom_gene,True,48237.277344,90649.484375,29386.843931,53712.358921
131578069,281646775406687,UNASSIGNED,0,BAX,10250.937500,19730.250000,13.937500,31.00,AC18,112.437500,215,custom_gene,True,48239.703125,92848.234375,29388.972927,55150.641494
131578070,281646775430168,UNASSIGNED,0,YTHDF2,10250.781250,19712.109375,13.859375,40.00,AC18,115.046875,135,custom_gene,True,48238.968750,92762.867188,29388.471494,55094.799861
131578072,281646775430169,UNASSIGNED,0,YTHDF2,10250.640625,20099.406250,13.828125,25.25,AC18,236.515625,135,custom_gene,True,48238.308594,94585.437500,29388.489101,56287.010239


In [27]:
filtered_transcripts['x'] = round(filtered_transcripts['x_transformed']).astype(int)
filtered_transcripts['y'] = round(filtered_transcripts['y_transformed']).astype(int)
filtered_transcripts.drop('cell_id', axis=1, inplace=True)

/tmp/ipykernel_61122/2966028094.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_transcripts['x'] = round(filtered_transcripts['x_transformed']).astype(int)
/tmp/ipykernel_61122/2966028094.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_transcripts['y'] = round(filtered_transcripts['y_transformed']).astype(int)
/tmp/ipykernel_61122/2966028094.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:

In [ ]:
mask_dir = "path/to/cellpose/output/morphometrics/2_masks/soma/tiles"
unique_features = filtered_transcripts["feature_name"].unique()

filtered_transcripts = filtered_transcripts.copy()
filtered_transcripts["x"] = filtered_transcripts["x"].round().astype(int)
filtered_transcripts["y"] = filtered_transcripts["y"].round().astype(int)

output_dir = "cellpose_transcripts_opposite_lyso"
os.makedirs(output_dir, exist_ok=True)

all_counts = []

for fov in range(1, 481):
    fname = f"W0001F{fov:04d}T0001Z001C1_seg.png"
    mask_arr = np.array(Image.open(os.path.join(mask_dir, fname)))

    if fov % 15 != 0:
        row_img = fov // 15 + 1
        col_img = fov % 15
    else:
        row_img = fov // 15
        col_img = 15

    x_off = int((col_img - 1) * 2000 * 0.98)
    y_off = int((row_img - 1) * 2000 * 0.98)

    rows = []
    for mask_id in np.unique(mask_arr):
        if mask_id == 0:
            continue
        y_pix, x_pix = np.where(mask_arr == mask_id)
        x_global = x_pix + x_off
        y_global = y_pix + y_off

        df = pd.DataFrame({
            "cell_id": f"{fov}_{mask_id}",
            "x_r":     np.round(x_global).astype(int),
            "y_r":     np.round(y_global).astype(int),
        })
        rows.append(df)
    if rows == []:
        print(f"FOV {fov} is empty")
        continue
    
    masks_df = pd.concat(rows, ignore_index=True)

    merged = (
        pd.merge(
            filtered_transcripts,
            masks_df,
            left_on=["x","y"],
            right_on=["x_r","y_r"],
            how="inner"
        )
        [["cell_id","feature_name"]]
    )

    counts = (
        merged
          .groupby(["cell_id","feature_name"])
          .size()
          .unstack(fill_value=0)
          .reindex(columns=unique_features, fill_value=0)
    )

    outpath = os.path.join(output_dir, f"counts_FOV{fov:04d}.csv")
    counts.to_csv(outpath)
    print(f"Finished FOV {fov}, saved to {outpath}")
print("All FOVs processed. Combined counts saved to combined_counts.csv")


Finished FOV 1, saved to cellpose_transcripts_opposite_lyso/counts_FOV0001.csv
Finished FOV 2, saved to cellpose_transcripts_opposite_lyso/counts_FOV0002.csv
Finished FOV 3, saved to cellpose_transcripts_opposite_lyso/counts_FOV0003.csv
Finished FOV 4, saved to cellpose_transcripts_opposite_lyso/counts_FOV0004.csv
Finished FOV 5, saved to cellpose_transcripts_opposite_lyso/counts_FOV0005.csv
Finished FOV 6, saved to cellpose_transcripts_opposite_lyso/counts_FOV0006.csv
Finished FOV 7, saved to cellpose_transcripts_opposite_lyso/counts_FOV0007.csv
Finished FOV 8, saved to cellpose_transcripts_opposite_lyso/counts_FOV0008.csv
Finished FOV 9, saved to cellpose_transcripts_opposite_lyso/counts_FOV0009.csv
Finished FOV 10, saved to cellpose_transcripts_opposite_lyso/counts_FOV0010.csv
Finished FOV 11, saved to cellpose_transcripts_opposite_lyso/counts_FOV0011.csv
Finished FOV 12, saved to cellpose_transcripts_opposite_lyso/counts_FOV0012.csv
Finished FOV 13, saved to cellpose_transcripts_op

Finished FOV 105, saved to cellpose_transcripts_opposite_lyso/counts_FOV0105.csv
Finished FOV 106, saved to cellpose_transcripts_opposite_lyso/counts_FOV0106.csv
Finished FOV 107, saved to cellpose_transcripts_opposite_lyso/counts_FOV0107.csv
Finished FOV 108, saved to cellpose_transcripts_opposite_lyso/counts_FOV0108.csv
Finished FOV 109, saved to cellpose_transcripts_opposite_lyso/counts_FOV0109.csv
Finished FOV 110, saved to cellpose_transcripts_opposite_lyso/counts_FOV0110.csv
Finished FOV 111, saved to cellpose_transcripts_opposite_lyso/counts_FOV0111.csv
Finished FOV 112, saved to cellpose_transcripts_opposite_lyso/counts_FOV0112.csv
Finished FOV 113, saved to cellpose_transcripts_opposite_lyso/counts_FOV0113.csv
Finished FOV 114, saved to cellpose_transcripts_opposite_lyso/counts_FOV0114.csv
Finished FOV 115, saved to cellpose_transcripts_opposite_lyso/counts_FOV0115.csv
Finished FOV 116, saved to cellpose_transcripts_opposite_lyso/counts_FOV0116.csv
Finished FOV 117, saved to c

Finished FOV 207, saved to cellpose_transcripts_opposite_lyso/counts_FOV0207.csv
Finished FOV 208, saved to cellpose_transcripts_opposite_lyso/counts_FOV0208.csv
Finished FOV 209, saved to cellpose_transcripts_opposite_lyso/counts_FOV0209.csv
Finished FOV 210, saved to cellpose_transcripts_opposite_lyso/counts_FOV0210.csv
Finished FOV 211, saved to cellpose_transcripts_opposite_lyso/counts_FOV0211.csv
Finished FOV 212, saved to cellpose_transcripts_opposite_lyso/counts_FOV0212.csv
Finished FOV 213, saved to cellpose_transcripts_opposite_lyso/counts_FOV0213.csv
Finished FOV 214, saved to cellpose_transcripts_opposite_lyso/counts_FOV0214.csv
Finished FOV 215, saved to cellpose_transcripts_opposite_lyso/counts_FOV0215.csv
Finished FOV 216, saved to cellpose_transcripts_opposite_lyso/counts_FOV0216.csv
Finished FOV 217, saved to cellpose_transcripts_opposite_lyso/counts_FOV0217.csv
Finished FOV 218, saved to cellpose_transcripts_opposite_lyso/counts_FOV0218.csv
Finished FOV 219, saved to c

# Find Morphology

In [5]:
data_dir = Path("cellpose_transcripts_opposite_lyso")

all_files = sorted(data_dir.glob("*.csv"))

dfs = [pd.read_csv(f, index_col=0) for f in all_files]

combined_df = pd.concat(dfs, axis=0)

In [7]:
top4 = ['ID2-4', 'SOX5-3', 'NFIB-2', 'ID2-2']
sgRNA_genes = [gene for gene in combined_df.columns if '-' in gene and not 'HLA' in gene]
sgRNA_cols = [
    col for col in sgRNA_genes
    if col not in top4
]

gene_names = sorted({col.split("-", 1)[0] for col in sgRNA_cols})
perturb_df = pd.DataFrame(index=combined_df.index, columns=gene_names, dtype="object")
for gene in gene_names:
    matching_cols = [col for col in sgRNA_cols if col.startswith(gene + "-")]
    if not matching_cols:
        continue

    sub_df = combined_df[matching_cols]
    any_positive = (sub_df > 0).any(axis=1)
    perturb_df[gene] = any_positive.map({True: "perturbed", False: "unperturbed"})

In [8]:
perturb_df

,AFF1,BCL6,BNC2,CARHSP1,CREB5,EBF1,EBF3,HMGB2,ID2,ID4,...,NFE2L2,NFIB,Negative_control,ONECUT1,ONECUT2,PROX1,RUNX1,SIX1,SOX5,ZBTB7C
cell_id,,,,,,,,,,,,,,,,,,,,,
1_1,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,...,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed
1_10,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,...,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed
1_11,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,...,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed
1_12,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,...,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed
1_13,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,...,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480_95,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,...,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed
480_96,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,...,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed
480_97,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,...,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed,unperturbed
